In [1]:
import json
import os
import copy
import torch
import pickle

from dataclasses import dataclass

from batchgenerators.utilities.file_and_folder_operations import load_json

from fvcore.nn import FlopCountAnalysis
from fvcore.nn import parameter_count_table

In [2]:
from mbas.tasks.predict import initialize_model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


2024-09-21 11:33:39.514919: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-21 11:33:39.537238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 11:33:39.905945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [36]:

model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_27__ResEncUNet_3d_lowres_for25_drop50_slim96"
model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__nnUNetResEncUNetMPlans_2024_08_10__3d_fullres"
model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__nnUNetResEncUNetMPlans_2024_08_10__3d_lowres"


model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_21__MedNeXtV2_3d_lowres_slim_96"
model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_21__MedNeXtV2_3d_lowres_foreground25"


model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_24__ResEncUNet_3d_lowres_for25"
model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_24__ResEncUNet_3d_lowres_k5_for25"


model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_27__ResEncUNet_3d_lowres_for25_drop50_slim128"
model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_27__ResEncUNet_3d_lowres_for25_drop50_slim96"


model_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_27__MedNeXtV2_3d_lowres_for25_drop50_stemStacked_decoderConvTrans"



checkpoint=torch.load(os.path.join(model_dir, "fold_0", "checkpoint_final.pth"), map_location=torch.device('cpu'))
dataset_json=load_json(os.path.join(model_dir, "dataset.json"))
plans_json = load_json(os.path.join(model_dir, "plans.json"))

model = initialize_model(
    checkpoint,
    dataset_json,
    plans_json,
    postprocessing_kwargs=[],
    compiled_model=False,
    skip_postprocessing=True
)

Network initialized: <class 'mbas.architectures.MedNeXtV2.MedNeXtV2'>


In [37]:
# image = torch.zeros((1, 1, 20, 256, 256))
# image = torch.zeros((1, 1, 32, 384, 384))

image = torch.zeros((1, 1, 28, 256, 224))
flops = FlopCountAnalysis(model.network, image)
flops_exp = flops.total() / 1e9
print(f"# FLOPs: {flops_exp:.1f}x10^9")

Unsupported operator aten::feature_dropout_ encountered 47 time(s)
Unsupported operator aten::leaky_relu_ encountered 1 time(s)
Unsupported operator aten::gelu encountered 46 time(s)
Unsupported operator aten::add_ encountered 46 time(s)
Unsupported operator aten::pad encountered 5 time(s)
Unsupported operator aten::add encountered 5 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
decoder.seg_layers.0, decoder.seg_layers.1, decoder.seg_layers.2, decoder.seg_layers.3


# FLOPs: 223.2x10^9


In [38]:
num_params =count_parameters(model.network)
print(f"# params: {num_params / 1e6:.1f}M")
print(parameter_count_table(model.network, max_depth=1))

# params: 28.3M
| name     | #elements or shape   |
|:---------|:---------------------|
| model    | 28.3M                |
|  encoder |  13.4M               |
|  decoder |  14.9M               |
